In [ ]:
%pip install netCDF4

In [ ]:
import netCDF4 as nc
import numpy as np
import matplotlib.pylab as py
from sklearn.linear_model import LinearRegression
from numpy.polynomial import polynomial as P

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
filename = '/content/drive/MyDrive/Teaching/25F - GEOG256 - RegClim Terrestrial Process/Data_code_folder/climate_warming_lecture2/tas_glob_piControl.nc'
dss = nc.Dataset(filename)
#print(dss['tasglob'])
taspi = dss['tasglob'][:]
taspim = np.average(taspi,axis=1)
print(taspim)
filename = '/content/drive/MyDrive/Teaching/25F - GEOG256 - RegClim Terrestrial Process/Data_code_folder/climate_warming_lecture2/tas_glob_abpt4CO2.nc'
ds = nc.Dataset(filename)
#print(ds)
for dim in ds.dimensions.values():
    print(dim)
#print(ds['tasglob'])
tas0 = ds['tasglob'][:]
Y = [0, 1, 2, 3, 4, 6, 7]
tas = tas0[Y,:]
taspim0 = taspim[Y]
print(taspim0)
tas00 = tas0[Y,:] #original air temperature
#print(tas00)
for nn in range(0,7):
    tas[nn,:] = tas[nn,:]-taspim0[nn]

#print(np.average(tas))
date1 = np.arange(0.5, 150.5, 1.0)
#print(date1)

In [ ]:
py.figure(1,figsize=(8, 6), dpi=80)
labname= ["BCC-CSM2-MR","CanESM5","CESM2","CNRM-ESM2-1","IPSL-CM6A-LR","UKESM1-0-LL","MPI-ESM1-2-LR"]
colorname= ["red","darkorange","royalblue","magenta","blue","seagreen","turquoise"]
for nn in range(0,7):
  py.plot(date1, tas[nn,:], color= colorname[nn], label= labname[nn])
py.legend(loc="lower right")
py.title("4 x CO2 global surface air temperature in CMIP6",fontsize=16)
py.xlim(0,150)
py.ylim(0,9)
py.ylabel("Global Temperature (oC)",fontsize=16)
py.xlabel("Year", fontsize=16)
py.rc("xtick",labelsize=14)
py.rc("ytick",labelsize=14)
py.show()

In [ ]:
#print(tas)
tasmmm = np.average(tas[:,:],axis=0)
#print(tasmmm)
py.figure(1,figsize=(8, 6), dpi=80)
py.plot(date1, tasmmm, color= 'black', label= 'CMIP6mean')
py.legend(loc="lower right")
py.title("4 x CO2 global surface air temperature in CMIP6",fontsize=16)
py.xlim(0,150)
py.ylim(0,9)
py.ylabel("Global Temperature (oC)",fontsize=16)
py.xlabel("Year", fontsize=16)
py.rc("xtick",labelsize=14)
py.rc("ytick",labelsize=14)
py.show()

In [ ]:
# get the curve fit regression parameters
t          = np.arange(0.5, 150.5, 1.0) # time in years
impulselen = len(t)
t          = np.log(t)
tasresponse = np.log(tasmmm)
K = P.polyfit(t,tasresponse,3)
print(K)
tresponse  = np.exp(K[0] + K[1]*t + K[2]*t**2 + K[3]*t**3) #fit to powerpoint curv

py.figure(1,figsize=(8, 6), dpi=80)
py.plot(date1, tasmmm, color= 'black', label= 'CMIP6mean')
py.plot(date1, tresponse, color= 'red', dashes = [3,1], label= 'CMIP6mean-fit')
#py.plot(date1, tasresponse, color= 'blue', dashes = [3,1], label= 'log')
py.legend(loc="lower right")
py.title("4 x CO2 global surface air temperature in CMIP6",fontsize=16)
py.xlim(0,150)
py.ylim(0,9)
py.ylabel("Global Temperature (oC)",fontsize=16)
py.xlabel("Year", fontsize=16)
py.rc("xtick",labelsize=14)
py.rc("ytick",labelsize=14)
py.show()

\begin{equation}
N = F + \lambda \Delta T
\end{equation}
Based on the equation above, compute the TOA energy imblance regression with $\Delta T$
TOA energy imbalance N, $\Delta T$ is derived from all 150 years simulation
They have been regressed against each other as shown below to compuate Equilibrium Climate Sensitivity (ECS)

In [ ]:
filename1 = '/content/drive/MyDrive/Teaching/25F - GEOG256 - RegClim Terrestrial Process/Data_code_folder/climate_warming_lecture2/rsdt_glob_abpt4CO2.nc'
filename2 = '/content/drive/MyDrive/Teaching/25F - GEOG256 - RegClim Terrestrial Process/Data_code_folder/climate_warming_lecture2/rsut_glob_abpt4CO2.nc'
filename3 = '/content/drive/MyDrive/Teaching/25F - GEOG256 - RegClim Terrestrial Process/Data_code_folder/climate_warming_lecture2/rlut_glob_abpt4CO2.nc'
ds1 = nc.Dataset(filename1)
ds2 = nc.Dataset(filename2)
ds3 = nc.Dataset(filename3)

rad0 = ds1['rsdtglob'][:]
Y = [0, 1, 2, 3, 4, 6, 7]
rsdt = rad0[Y,:]
rad0 = ds2['rsutglob'][:]
rsut = rad0[Y,:]
rad0 = ds3['rlutglob'][:]
rlut = rad0[Y,:]
netrad = rsdt-rsut-rlut

Gregory regression method paper:
https://doi.org/10.1029/2003GL018747

In [ ]:
# Gregory regression
labname= ["BCC-CSM2-MR","CanESM5","CESM2","CNRM-ESM2-1","IPSL-CM6A-LR","UKESM1-0-LL","MPI-ESM1-2-LR"]
regall = np.zeros((7,2))
for nn in range(0,7):
  reg = LinearRegression().fit(tas[nn,:].reshape((150,1)),netrad[nn,:].reshape((150,1)))
  #print(reg.coef_)
  #print(reg.intercept_)
  regall[nn,0] = reg.coef_[0,0]  # a*X0+b
  regall[nn,1] = reg.intercept_[0]
#print(regall)

x0 = np.arange(0.,12.1,1)
yy0 = np.arange(0.,12.1,1)
y0 = np.zeros((7,13))
ECS1 = np.zeros(7)
for nn in range(0,7):
  y0[nn,:] = x0*regall[nn,0]+regall[nn,1]
  print("Equilibrium climate sensitivity (ECS) for ",labname[nn],":")
  print(regall[nn,1]*-1/regall[nn,0]/2)
  ECS1[nn] = regall[nn,1]*-1/regall[nn,0]/2
ECS2 = [3.0, 5.6, 5.2, 4.3, 4.6, 5.3, 3.0]
py.figure(1,figsize=(8, 6), dpi=80)
labname= ["BCC-CSM2-MR","CanESM5","CESM2","CNRM-ESM2-1","IPSL-CM6A-LR","UKESM1-0-LL","MPI-ESM1-2-LR"]
colorname= ["red","darkorange","royalblue","magenta","blue","seagreen","turquoise"]
for nn in range(0,7):
  py.scatter(ECS1[nn],ECS2[nn], color= colorname[nn], label= labname[nn])
py.plot(x0,yy0,dashes=[3,1])
py.legend(loc="lower right")
py.title("Comparison of ECS with literature",fontsize=16)
py.xlim(0,7)
py.ylim(0,7)
py.xlabel("ECS calculated here (oC)",fontsize=16)
py.ylabel("ECS from Meehl et al. 2020 (oC)", fontsize=16)
py.rc("xtick",labelsize=14)
py.rc("ytick",labelsize=14)
py.show()
#print(y0)

In [ ]:
# plot the d_N vs d_T
#print(tas)
py.figure(1,figsize=(8, 6), dpi=80)
labname= ["BCC-CSM2-MR","CanESM5","CESM2","CNRM-ESM2-1","IPSL-CM6A-LR","UKESM1-0-LL","MPI-ESM1-2-LR"]
colorname= ["red","darkorange","royalblue","magenta","blue","seagreen","turquoise"]
for nn in range(0,7):
  py.scatter(tas[nn,:],netrad[nn,:], color= colorname[nn], label= labname[nn])
  py.plot(x0,y0[nn,:],color= colorname[nn] )
py.legend(loc="upper right")
py.title("Gregory regression for CMIP6",fontsize=16)
py.axhline(y=0,color ="black",dashes=[3,2],linewidth=0.8)
py.xlim(0,12)
py.ylim(-2, 10)
py.xlabel("Global Temperature (oC)",fontsize=16)
py.ylabel("TOA net radiation (W/m-2)", fontsize=16)
py.rc("xtick",labelsize=14)
py.rc("ytick",labelsize=14)
py.show()

Based on the text in https://doi.org/10.1029/2019GL085782:
Try to estimate and print
1) "effective radiative forcing" (y-intercept divided by 2)
2) "radiative feedback" (regression slope)
3) "effective climate sensitivity" (x-intercept divided by 2)
"The division by 2 is done so as to express ERF and ECS with respect to a CO2 doubling, as is standard practice." - Zelinka et al. (2020)


In [ ]:
# Your code and results here and after